In [1]:
import numpy as np
import random
from scipy.spatial import distance

n_dimensions=3
n_points=100

#Create samples on a unit sphere in desired dimensions
def hypersphere(npoints, ndim):
    np.random.seed(1)
    vec=np.random.randn(ndim,npoints) #creates a random sample from a Gaussian distribution in the form of an array of dimensions: ndim x npoints
    vec/=np.linalg.norm(vec,axis=0) #divides each vector by its norm, which turns each vector into a unit vector (length 1). Here we obtain samples from the unit sphere in the dimension we stated in the beginning of the function.
    return(vec)

In [2]:
#Randomly select a point on the hypersphere to be the center of the ball
random.seed(1)
hsphere=hypersphere(n_points,n_dimensions)
dim_array=np.shape(hsphere) #defines the shape of the array
rng=random.randint(0,dim_array[1])#pulls random number from 0 to n_points and assign to rng
center_ball=hsphere[:,rng] #uses randomly generated number to extract one row of points
print(center_ball)
print(hsphere[0:2,0])

[-0.46457661 -0.85509238 -0.23018598]
[ 0.93795301 -0.25818745]


In [3]:
radii = np.arange(.1,1.01,.01) #creates an array of radii values to iterate through

#Find the points on the hypersphere within each radii
for i in radii: #iteratest through each radii value
    X=np.empty([0,dim_array[0]]) #creates an empty array to be filled in with the values on the hypersphere that fall within the ball radii
    for j in np.arange(0,n_points): #iterates through array of index values for each point
        if distance.euclidean(hsphere[:,j],center_ball)<=i: #selects points whose distance to center is shorter than the length of the radius
            if np.shape(X)==(0,dim_array[0]):
                X=np.append(X,hsphere[:,j]) #appends first point coordinates to the empty array
            else:
                X=np.vstack((X,hsphere[:,j])) #vertically stacks subsequent points to the empty array
    dim_X=np.shape(X) #saves dimensions of matrix for points within the current radius
    #Need to set up covariance matrix within this and perform PCA

In [4]:
distance.euclidean(hsphere[:,22],center_ball)

1.921649874320172

In [5]:
x=np.empty([0,3])

In [6]:
a=[1,2,3]
x=np.vstack((x,a))
x

array([[1., 2., 3.]])

In [151]:
np.shape(x)

(0, 3)

<h1>Ryan's Comments</h1>
<p>
    Hey Kyle! Here's me presenting an idea as to how you could approach the covariance matrix. Let $X_c$ be the set of points within some radius $\epsilon$ of center point $c$. We assume that $X_c$ consists of $N$ points in $d$ dimensions, and we represent it as a $N\times d$ matrix.
    <br/><br/>
    In the block below, I generate $X_c$ as a random $N\times d$ matrix, just so that the cell below runs.
</p>

In [14]:
# Initialize N, d, X_c
N = 500
d = 20
X_c = np.random.randn(N, d)
print(X_c.shape)

# Center points at their mean
# axis = 0 makes sure that the mean of each column is taken from all rows
mean = np.mean(X_c, axis=0)
X_cc = X_c - mean

# For now, we assume that our data need not be normalized
# For 2-dimensional NumPy arrays, "@" defaults to the standard matrix product 
covariance = 1/N * (X_cc.T @ X_cc)

# Get eigenvalues and eigenvectors through np.linalg.eigh
eigvals, eigvecs = np.linalg.eigh(covariance)

# Eigenvalues should be sorted in increasing order
print(eigvals)

(500, 20)
[-0.02763406 -0.03523797  0.01091091  0.02743391  0.05306933 -0.04435852
 -0.0190819   0.02641563  0.04160879 -0.0869273   0.0598225  -0.0144549
  0.01147376  0.0520233  -0.0521974   0.01076213  0.07494473 -0.05430685
  0.00947868 -0.02640565]
(500, 20)
(20, 20)
[0.61987605 0.7127197  0.73361923 0.76811644 0.80635459 0.81289635
 0.84781395 0.87373054 0.93386579 0.95929094 0.98435485 1.0375708
 1.06086397 1.1221724  1.16084316 1.17381874 1.21177891 1.26589367
 1.2784113  1.33270012]
